In [32]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

In [33]:
def social_equity_m_1_data(scenario_id):
    conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=abm_14_2_0_reporting;'
                    'Trusted_Connection=yes;')

    query =f'''-- connect to ddamwsql2016
    SELECT [scenario_id]
        ,[measure]
        ,[metric]
        ,ROUND([value]*100,1) as [rounded_val] -- change this depending on value, i.e. if percent or other value
        ,[value] as [og_val]
        ,[updated_by]
        ,[updated_date]
    FROM [abm_14_2_0_reporting].[rp_2021].[results]
    WHERE scenario_id = {scenario_id}
    AND measure like 'M-1%'
    '''

    sql_data =  pd.read_sql_query(query, conn)

    lst = list(sql_data['metric'])

    lst = [s.split('-') for s in lst]

    first_values = [inner_lst[0] if inner_lst[0] != 'Non' else inner_lst[0]+'-'+inner_lst[1] for inner_lst in lst]
    second_values = [inner_lst[1] if inner_lst[0] != 'Non' else inner_lst[2] for inner_lst in lst]

    sql_data['population'] = first_values
    sql_data['mode'] = second_values

    sql_data['mode'] = sql_data['mode'].replace({' Bike':'Bike', 
                                                ' Bike + Walk + MM + MT':'Walk, Bike, Micromobility, Microtransit',
                                                ' Drive Alone':'Driving (drive alone)',
                                                ' Transit ': 'Transit - Accessed by Walk and or Flexible Fleet - Speed One',
                                                ' Walk':'Walk',
                                                ' Walk + MM + MT':'Walk, Micromobility, Microtransit'
                                                })

    output_rank= {'Bike':2, 
                                                'Walk, Bike, Micromobility, Microtransit':4,
                                                'Driving (drive alone)':6,
                                                'Transit - Accessed by Walk and or Flexible Fleet - Speed One':5,
                                                'Walk':1,
                                                'Walk, Micromobility, Microtransit':3
                                                }

    sql_data['output_ranking'] = [output_rank[mode] for mode in sql_data['mode']]

    final_output = sql_data.sort_values('output_ranking')

    measures = ['M-1-a','M-1-b','M-1-c']

    pops = ['Low Income Access Pct ','Non-Low Income Access Pct ','Minority Access Pct ','Non-Minority Access Pct ','Senior Access Pct ','Non-Senior Access Pct ']

    concatonated_dfs = pd.DataFrame()

    for measure in measures:
        for ind_pop in pops:
            #print(measure + ' - ' + ind_pop + ' -  Regional')
            df = final_output[final_output['measure'] == measure]
            df = df[df['population'] == ind_pop]
            df['measure'] = measure
            df['Region or Mohub'] = 'Regional'
            concatonated_dfs = pd.concat([concatonated_dfs, df[['measure', 'Region or Mohub', 'population', 'mode', 'rounded_val']]])
            #print(df[['measure', 'population', 'mode', 'rounded_val']])

            #print(measure + ' - ' + ind_pop + ' -  Mohub')
            df = final_output[final_output['measure'] == measure+' - Mobility Hubs']
            df = df[df['population'] == ind_pop]
            df['measure'] = measure
            df['Region or Mohub'] = 'Mohub'
            concatonated_dfs = pd.concat([concatonated_dfs, df[['measure', 'Region or Mohub', 'population', 'mode', 'rounded_val']]])
            #print(df[['measure', 'population', 'mode', 'rounded_val']])
    
    return concatonated_dfs.groupby(['measure', 'Region or Mohub', 'population', 'mode']).sum()

In [34]:
test = social_equity_m_1_data(scenario_id=768)

In [35]:
test

rounded_val
measure Region or Mohub population               mode                                                           
M-1-a   Mohub           Low Income Access Pct    Bike                                                      100.0
                                                 Driving (drive alone)                                     100.0
                                                 Transit - Accessed by Walk and or Flexible Flee...         93.4
                                                 Walk                                                       96.1
                                                 Walk, Bike, Micromobility, Microtransit                   100.0
...                                                                                                          ...
M-1-c   Regional        Non-Minority Access Pct  Transit - Accessed by Walk and or Flexible Flee...         81.4
                        Non-Senior Access Pct    Driving (drive alone)                                     100.0
                                                 Transit - Accessed by Walk and or Flexible Flee...         85.5
                        Senior Access Pct        Driving (drive alone)                                     100.0
                                                 Transit - Accessed by Walk and or Flexible Flee...         84.5

[168 rows x 1 columns]